# Curso de Python - Desafio 1

## Importações

In [101]:
import csv
import psycopg2 as p

## Função para ler arquivo CSV

In [102]:
def ler_csv(arquivo_csv, sep): 
    with open(arquivo_csv, "r") as arq_csv: # abre arquivo .csv somente leitura
        next(arq_csv) # pula a primeira linha
        leitor_csv = csv.reader(arq_csv, delimiter=sep) # cria o objeto csv iterator   
        lista = [x for x in leitor_csv] # percorre objeto csv iterator para carregar em uma lista 
    return lista    

## Função para gravar no banco

In [103]:
def gravar_banco(lista):
    # conecta banco Postgres Amazon RDS
    con = p.connect(host="dsadev.cjzzjgprwiev.us-east-2.rds.amazonaws.com", port="5432", database="devdb", user="postgres", password="asc210996")
    con.autocommit = False # desabilita auto commit
    cur = con.cursor() # carrega cursor
    
    for i in lista: # percorre a lista passada como parâmetro
        sql = "INSERT INTO curso_python_desafio1 (ID,LONG,LAT,SETCENS,AREAP,CODDIST,DISTRITO,CODSUBPREF,SUBPREF,REGIAO5,REGIAO8,ESTABELECI,ENDERECO,BAIRRO,TELEFONE,CEP,CNES,SA_DEPADM,DEPADM,SA_TIPO,TIPO,SA_CLASSE,CLASSE,LEITOS) VALUES ("
        sql = sql + "'" + "'".join(map(lambda x: x + "',", i)) # concatena a lista percorrida para formar o comando insert
        sql = sql[:-1] + ")" # retira o último caractere que é uma vírgula e adiciona o parêntese
        
        try:
            cur.execute(sql) # executa comando sql insert
            con.commit() # commit de cada insert
        except Exception as e:
            con.rollback() # rollback insert em caso de erro
            
            with open("curso-python-desafio1.log", "w") as log: # grava em log no caso de erro no insert
                log.write(sql + "\n" + str(e) + "\n")
            
            pass # não para quando ocorrer erro
    
    cur.close() # fecha cursor
    con.close() # fecha conexão
                

In [104]:
gravar_banco(ler_csv("deinfosacadsau2014.csv", ","))